# Visualize TimePix3 data

read from "hdf5" files created by Kuepper et al. (Hubertus B.)

### Imports

In [1]:
# %matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

plt.rc('figure', figsize=(20, 10))


### Define file

In [3]:
hdf_file_name = '../data/20191031-052753_CH3I_000000.hdf5'

### Open HDF file

In [4]:
h_file = h5py.File(hdf_file_name,'r')

### Read from HDF file

In [5]:
tof = h_file['centroided/tof'][:]
x_pos = h_file['centroided/x'][:]
y_pos = h_file['centroided/y'][:]

### Utility function

In [6]:
def generate_plots(tof, x_pos, y_pos, hist_bins='auto'):
    
    fig = plt.figure()
    plt.hist(tof, bins= hist_bins)
    plt.title('histogram: time-of-flight')
    plt.xlabel('ToF [s]')
    plt.ylabel('number of events')

    fig = plt.figure()
    plt.plot(tof, x_pos,'.')
    plt.title("ToF vs x_pos")
    plt.xlabel('ToF [s]')
    plt.ylabel('x_pos [px]')

    fig = plt.figure()
    plt.plot(tof, y_pos,'.')
    plt.title("ToF vs y_pos")
    plt.xlabel('ToF [s]')
    plt.ylabel('y_pos [px]')

    plt.show()
    

In [ ]:
%%time 

generate_plots(tof, x_pos, y_pos)

### Remove events from outside the measurement window 

In [ ]:
cutoff = 0.0012
x_pos = x_pos[tof < cutoff]
y_pos = y_pos[tof < cutoff]
tof = tof[tof < cutoff]

generate_plots(tof, x_pos, y_pos, hist_bins = 10000)

### Reduce data to one agglomeration

In [ ]:
cutoff = 0.0002
x_pos = x_pos[tof < cutoff]
y_pos = y_pos[tof < cutoff]
tof = tof[tof < cutoff]

generate_plots(tof, x_pos, y_pos, hist_bins = 1000)

### Reduce data to one single FEL train

In [ ]:
cutoff = 1E-6
x_pos = x_pos[tof < cutoff]
y_pos = y_pos[tof < cutoff]
tof = tof[tof < cutoff]

generate_plots(tof, x_pos, y_pos, hist_bins = 1000)